# 1-Webcrawller Information
Date created: 13-04-2023  
Created by: Jayden Dzierbicki  
Last updated: 13-04-2023  

The purpose of this script is to obtain data associated with XRP historic price, and XRP forum posts. We do this by scrapping/crawling the following websites:
- https://finance.yahoo.com/quote/XRP-AUD/history?p=XRP-AUD Obtain XRP price in AUD
- https://www.cryptocompare.com/coins/xrp/forum Obtain forum posts for XRP

All data obtained will be stored in mySQL, with a CSV backup created as well

In [ ]:
#Requirments to work
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
import dateparser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
from datetime import date 



## cryptocompare.com crawler
- Step 1: Crawl and save CSV backup
- Step 2: Store data in mySQL

In [ ]:
# Step 1:
scroll_duration = timedelta(hours=0.25)  # Set the duration for scrolling

# Configure Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # More efficient RAM usage

# Replace with the path to your ChromeDriver
driver = webdriver.Chrome(executable_path='C:/webdrivers/chromedriver', options=chrome_options)

url = f"https://www.cryptocompare.com/coins/xrp/forum"
driver.get(url)

# Calculate the number of iterations for the progress bar
scroll_interval = 3  # seconds based on sleep time
total_iterations = int(scroll_duration.total_seconds() // scroll_interval)

# Initialize the progress bar
progress_bar = tqdm(range(total_iterations), desc="Scraping posts")

start_time = datetime.now()
post_data2 = []
prev_post_count = 0
iteration = 0
while iteration < total_iterations:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_interval)

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    posts = soup.find_all("div", {"class": "post-content"})

    if len(posts) > prev_post_count:  # Check if new posts are loaded
        for post in posts[prev_post_count:]:
            comment_element = post.find("div", {"class": "content-body"})
            date_element = post.find("div", {"class": "item-ago ng-binding"})
            if comment_element and date_element:
                date_str = date_element['title']
                parsed_date = dateparser.parse(date_str, settings={"RELATIVE_BASE": datetime.now()})
                comment = comment_element.text.replace('\n', '').strip()
                post_data2.append({"comment": comment, "date": parsed_date})

        prev_post_count = len(posts)

    iteration += 1
    progress_bar.update(1)  # Update the progress bar

driver.quit()
progress_bar.close()  # Close the progress bar

#df2 = pd.DataFrame(post_data2)


# Save DataFrame as CSV file - backup for SQL
#today_date = date.today()
#df2.to_csv(f'cryptocompare_{today_date}.csv', index=False)
#df2

In [ ]:
# Step 2
# Add an ID column to the DataFrame
df['id'] = df.index + 1

# Create the connection to the MySQL database using sqlalchemy
user = 'root'

# Prompt the user for a password
password = getpass("Enter your MySQL password: ")
host = 'localhost'
port = 3306
database = 'MA5851_A3'

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# Write the DataFrame to a SQL table
table_name = 'cryptocompare_xrp'
df.to_sql(table_name, engine, if_exists='replace', index=False)

# Close the connection
engine.dispose()

## Yahoofinance.com scrapper
- Step 1: Crawl and save CSV backup
- Step 2: Store data in mySQL

In [ ]:
# Step 2
# Add an ID column to the DataFrame
df['id'] = df.index + 1

# Create the connection to the MySQL database using sqlalchemy
user = 'root'

# Prompt the user for a password
password = getpass("Enter your MySQL password: ")
host = 'localhost'
port = 3306
database = 'MA5851_A3'

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# Write the DataFrame to a SQL table
table_name = 'cryptocompare_xrp'
df.to_sql(table_name, engine, if_exists='replace', index=False)

# Close the connection
engine.dispose()